<h2>Basketball Court Tracker - BCT</h2>

---

**Autores:** Óscar Muñoz Hidalgo y Juan José Quesada Acosta

**IMPORTANTE:** Ejecutar todas las celdas en orden para el correcto funcionamiento del programa.

---

<h3>Librerías y paquetes usados en el cuaderno</h3>

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.cluster import KMeans
from skimage import transform

import os
from datetime import datetime

<h3>Funciones para la detección de la zona de 3 segundos y la homografía</h3>

In [2]:
#Intersección entre dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height

# Desnormaliza los puntos de la zona recortada a los del frame original
def denormalizeCorners(corners, xmin, ymin):
    denormalized_corners = []
    for point in corners:
        x = int(point[0] + xmin)
        y = int(point[1] + ymin)
        denormalized_corners.append((x, y))
    return denormalized_corners

# Genera nubes de puntos cercanos a partir de las esquinas desnormalizadas
def groupCorners(denormalized_corners):
    corner_groups = []
    for i in range(len(denormalized_corners)):
        # Si no hay nubes de puntos, genera una nueva
        if(len(corner_groups) == 0):
            corner_groups.append([denormalized_corners[i]])
            continue

        # Revisa todas las nubes de puntos
        for j in range(len(corner_groups)):
            is_close = False
            for k in range(len(corner_groups[j])):
                # Si el punto está cerca de una nube, añadirlo a la nube
                if np.linalg.norm(np.array(denormalized_corners[i]) - np.array(corner_groups[j][k])) < 50:
                    corner_groups[j].append(denormalized_corners[i])
                    is_close = True
                    break
            if is_close:
                break

        # Si no encuentra una nube cercana, crea una nueva
        if not is_close:
            corner_groups.append([denormalized_corners[i]])

    return corner_groups

# Filtra las nubes de puntos para obtener las esquinas de la zona
def filterCorners(corner_groups, previous_zone_points, frame_width, frame_height):
    global is_right_side
    ordered_points = []
    xmin = (frame_width,0)
    ymin = (0,frame_height)
    xmax = (0,0)
    ymax = (0,0)

    # Encontrar las esquinas de la zona
    for i in range(len(corner_groups)):
        for j in range(len(corner_groups[i])):
            if(corner_groups[i][j][0] < xmin[0]):
                xmin = corner_groups[i][j]
            if(corner_groups[i][j][1] < ymin[1]):
                ymin = corner_groups[i][j]
            if(corner_groups[i][j][0] > xmax[0]):
                xmax = corner_groups[i][j]
            if(corner_groups[i][j][1] > ymax[1]):
                ymax = corner_groups[i][j]
    

    # Detectar en qué lado de la cancha está el aro en la primera ejecución
    if is_right_side is None:
        is_right_side = ymin[0] > ymax[0] and xmax[0] > frame_width*0.75

    # Lista de puntos extremos detectados, en orden A, B, C, D
    # Si es el lado derecho, el orden es xmin, ymax, ymin, xmax
    # Si es el lado izquierdo, el orden es xmax, ymax, ymin, xmin
    current_points = [xmin, ymax, ymin, xmax] if is_right_side else [xmax, ymax, ymin, xmin]

    # Ajustar los puntos basándose en la distancia a los puntos de la zona anterior detectados
    if len(previous_zone_points) > 0:
        
        for i, current_point in enumerate(current_points):
            previous_point = previous_zone_points[i]
            distance = np.linalg.norm(np.array(current_point) - np.array(previous_point))


            if distance < 20:
                # Mantener el punto actual para aumentar la estabilidad
                adjusted_point = previous_point

            elif 20 <= distance <= 30:
                # Suavizado exponencial
                adjusted_point = tuple(
                    0.9 * np.array(previous_point) + 0.1 * np.array(current_point)
                )

            elif 30 <= distance <= 50:
                # Suavizado exponencial
                adjusted_point = tuple(
                    0.85 * np.array(previous_point) + 0.15 * np.array(current_point)
                )
            else:
                adjusted_point = tuple(
                    0.7 * np.array(previous_point) + 0.3 * np.array(current_point)
                )

            ordered_points.append(adjusted_point)
    else:
        ordered_points = current_points
    

    return ordered_points, is_right_side

# Detección de las esquinas de la zona de juego a partir de la máscara de la zona de 3 segundos (clase 'paint')
def getZoneCorners(frame, model):

    # Realizar segmentación en el frame
    results = model.predict(frame, task='segment')

    # Dimensiones del frame original
    frame_height, frame_width = frame.shape[:2]

    # Se preestablecen las coordenadas del segmento a investigar
    xmin, ymin, xmax, ymax = 0,0,0,0
    cropped_zone = 0

    # Iterar sobre los resultados
    for result in results:
        masks = result.masks.data.cpu().numpy()  # Máscaras (NumPy array)
        classes = result.boxes.cls.cpu().numpy()  # Clases detectadas
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
        image = frame.copy()  # Crear copia del frame original para visualización

        # Aplicar y mostrar máscaras
        for i, cls in enumerate(classes):
            if model.names[cls] == 'paint': # Zona de 3 segundos
                # Redimensionar la máscara al tamaño del frame original
                zone_mask = (masks[i] * 255).astype(np.uint8)  # Escalar la máscara (0-255)
                resized_zone_mask = cv2.resize(zone_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

                # Invertir la máscara
                zone_mask_inv = cv2.bitwise_not(resized_zone_mask)

                # Aplicar la máscara al frame original
                combined = cv2.bitwise_and(image, image, mask=zone_mask_inv)
                
                # Recortar la zona de 3 segundos
                xmin, ymin, xmax, ymax = boxes[i].astype(int)
                cropped_zone = combined[ymin:ymax, xmin:xmax]


    # Detección de líneas en la zona designada
    image = cropped_zone.copy()

    # Preprocesamiento de la imagen
    gaussian_image = cv2.GaussianBlur(image, (17,17), 0)
    mask = cv2.inRange(gaussian_image, 0,1)
    gaussian_masked_frame = cv2.GaussianBlur(mask, (17,17), 0)
    canny_masked_frame = cv2.Canny(gaussian_masked_frame, 30, 150)
    bilateral_canny_masked_frame = cv2.bilateralFilter(canny_masked_frame, 6, 75, 75)

    # Detecta líneas con la transformada de Hough
    min_line_length = canny_masked_frame.shape[0]*0.5  # Longitud mínima de la linea
    lineas_campo = cv2.HoughLinesP(bilateral_canny_masked_frame, rho=1, theta=np.pi / 180, threshold=15, minLineLength=min_line_length, maxLineGap=105)

    # Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
    nlineas = 10
    if lineas_campo is not None:
        lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
        lineas_campo = lineas_campo[:10]

    # Inicializa grupos de líneas y umbral de orientación para separar
    group_1 = []  
    group_2 = []  
    angle_threshold = 20

    # Agrupa líneas por orientación
    if lineas_campo is not None:
        for line in lineas_campo:
            x1, y1, x2, y2 = line[0]
            
            # Evita división por cero en líneas verticales
            if x2 - x1 == 0:
                angle = 90  
            else:
                # Ángulo en grados
                angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
                angle = abs(angle) 
            
            # Asignar grupo
            if abs(angle) < angle_threshold:  # Cercano a horizontal
                group_1.append(line)
            else:  # Resto
                group_2.append(line)

    #INTERSECCIONES
    # Obtener las intersecciones entre líneas de grupo diferente
    intersections = []

    for i in range(len(group_1)):
        for j in range(len(group_2)):
            line1 = group_1[i][0]
            line2 = group_2[j][0]

            intersection = line_intersection(line1, line2)
            # Chequea que esté en la imagen
            if intersection:
                intersections.append(intersection)

    return intersections, xmin, ymin

# Función auxiliar para obtener el área de un polígono dados 4 puntos
def polygonArea(points):
    # Ensure there are exactly 4 points
    if len(points) != 4:
        raise ValueError("There must be exactly 4 points")

    # Shoelace formula
    x1, y1 = points[0]
    x2, y2 = points[1]
    x3, y3 = points[2]
    x4, y4 = points[3]

    # Fórmula de Shoelace
    area = 0.5 * abs(x1*y2 + x2*y3 + x3*y4 + x4*y1 - y1*x2 - y2*x3 - y3*x4 - y4*x1)
    return area

# Función para calcular la matriz de homografía a partir de las esquinas detectadas
def calculateHomography(zone_points, is_right_side):
    # Puntos del diagrama para la homografía: [A, B, C, D]
    diagram_points_right_side = [(1509, 436), (1509, 727), (1854, 436), (1854, 727)]
    diagram_points_left_side = [(490, 436), (490, 727), (145, 436), (145, 727)]

    if is_right_side:
        diagram_points = diagram_points_right_side
    else:
        diagram_points = diagram_points_left_side

    # Matriz de homografía
    return transform.estimate_transform('projective', np.array(zone_points), np.array(diagram_points))

# Función para dibujar las detecciones en el diagrama
def drawDetections(homography_matrix, diagram, team_A, team_B, referees):
    for point in team_A:
        diagram_point = homography_matrix(point)
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 15, getTeamBGRColor("A"), -1)

    for point in team_B:
        diagram_point = homography_matrix(point)
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 15, getTeamBGRColor("B"), -1)

    for point in referees:
        diagram_point = homography_matrix(point)
        cv2.rectangle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), (int(diagram_point[0][0]) + 15, int(diagram_point[0][1]) + 15), (0, 0, 0), -1)



<h3>Funciones para la detección de jugadores y árbitros, y asignación de equipos</h3>

In [3]:
# Función para obtener el ROI del área central de la bbox
def get_central_roi(frame, x1, y1, x2, y2):
    # Coordenadas de la bbox
    width = x2 - x1
    height = y2 - y1

    # Definir subárea como proporción de la bbox
    sub_x1 = int(x1 + width * 0.3)  # Dejar un 30% de margen a los lados
    sub_x2 = int(x2 - width * 0.3)  # Dejar un 30% de margen a los lados
    sub_y1 = int(y1 + height * 0.3)  # Subárea empieza al 30% de la altura
    sub_y2 = int(y1 + height * 0.6)  # Subárea llega al 60% de la altura

    # Recortar y devolver el ROI
    roi = frame[sub_y1:sub_y2, sub_x1:sub_x2]
    return roi

# Función para obtener los colores de los equipos
def classifyTeam(player_roi):
    
    # Convertir a HSV y preparar los datos para KMeans (matriz de (n_pixels, 3))
    roi_hsv = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)
    pixels = roi_hsv.reshape(-1, 3)

    # Aplicar KMeans para obtener los colores dominantes
    k = 2
    kmeans = KMeans(n_clusters=k, random_state=0).fit(pixels)
    dominant_colors = kmeans.cluster_centers_
    labels, counts = np.unique(kmeans.labels_, return_counts=True)

    # Obtener el color más dominante
    dominant_cluster = labels[np.argmax(counts)]
    dominant_color = dominant_colors[dominant_cluster]

    # Si no se han clasificado los colores de los equipos, asignar el color a uno de ellos
    if np.array_equal(team_colors['A'], np.array([0,0,0])):
        team_colors['A'] = dominant_color
        return 'A'
    
    # Si ya se ha clasificado el color de un equipo, asignar el color al otro
    elif np.array_equal(team_colors['B'], np.array([0,0,0])):
        # Si el color es muy similar al del equipo A, evita asignarlo a B
        if np.linalg.norm(dominant_color - team_colors['A']) < 100:
            return 'A'
        team_colors['B'] = dominant_color
        return 'B'
    
    # Si ya se han clasificado ambos colores, asignar el color al equipo más cercano
    else:
        if np.linalg.norm(dominant_color - team_colors['A']) < np.linalg.norm(dominant_color - team_colors['B']):
            return 'A'
        else:
            return 'B'

# Función para obtener el color BGR de un equipo a partir de su nombre
def getTeamBGRColor(team):
    if team not in team_colors:
        return [0, 0, 0]
    bgr_color = team_colors[team].astype(np.uint8)
    bgr_color = cv2.cvtColor(bgr_color[np.newaxis, np.newaxis, :], cv2.COLOR_HSV2BGR)[0, 0]
    return bgr_color.tolist()

# Función para dibujar las bbox en el frame
def drawBBox(frame, x1, y1, x2, y2, team, label):
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), getTeamBGRColor(team), 2)
    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 200, 100), 2)

# Función para dibujar la posición de un jugador en el frame
def drawPosition(frame, position, position_label):
    cv2.circle(frame, (int(position[0]), int(position[1])), 5, (0, 0, 255), -1)
    #cv2.ellipse(frame, (int(position[0]), int(position[1])), (15, 5), 0, 0, 360, (0, 0, 255), -1)
    cv2.putText(frame, position_label, (int(position[0]) - 50, int(position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Función para suavizar las posiciones de los jugadores con un filtro exponencial teniendo en cuenta las posiciones anteriores
def smooth_positions(current_positions, previous_positions, alpha=0.7, distance_threshold=50):

    # Si no hay posiciones anteriores, retornar las actuales sin cambios
    if not previous_positions:
        return current_positions

    smoothed_positions = []

    for cur_pos in current_positions:
        # Encontrar el punto más cercano en previous_positions
        closest_point = None
        min_distance = float('inf')

        for prev_pos in previous_positions:
            distance = np.linalg.norm(np.array(cur_pos) - np.array(prev_pos))
            if distance < min_distance:
                min_distance = distance
                closest_point = prev_pos

        # Aplicar suavizado exponencial si está dentro del umbral
        if closest_point and min_distance <= distance_threshold:
            smoothed_pos = (
                int(alpha * cur_pos[0] + (1 - alpha) * closest_point[0]),
                int(alpha * cur_pos[1] + (1 - alpha) * closest_point[1])
            )
        else:
            # Si no hay punto cercano, usar la posición actual
            smoothed_pos = cur_pos

        smoothed_positions.append(smoothed_pos)

    return smoothed_positions


<h3>Programa principal</h3>

Puede modificarse el vídeo usada por el programa cambiando la ruta en la variable **video_path**. Hay 5 vídeos de ejemplo en */assets/videos*.

Tambien puede modificarse el diagrama por uno en fondo negro con líneas blancas cambiando la ruta en la variable **diagram**. Su ruta es */assets/court/diagrams/black_court.png*

In [ ]:
# Cargar el modelo YOLO entrenado
detection_model = YOLO("./yolo_models/detection/bpdv1.pt")
segment_model = YOLO("./yolo_models/segmentation/segmentv2.pt")

# Ruta del vídeo de entrada y del diagrama
video_path = "./assets/videos/dal-lac1.mp4"
diagram = cv2.imread('./assets/court_diagrams/white_court.png')

# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


# CONFIGURACIONES PARA LOS VÍDEOS DE SALIDA
# Obtener el nombre del archivo sin la extensión
video_name = os.path.splitext(os.path.basename(video_path))[0]

# Obtener la fecha y hora actual en el formato requerido
current_time = datetime.now().strftime("%d-%m-%Y_%H-%M")

# Crear la ruta de la carpeta de salida
output_dir = f"./output/{video_name}_{current_time}"

# Crear la carpeta si no existe
os.makedirs(output_dir, exist_ok=True)

# Crear la ruta de salida con el formato deseado
output_path = f"./output/{video_name}_{current_time}/video.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Configurar el video de salida para el diagrama
diagram_output_path = f"./output/{video_name}_{current_time}/diagrama.mp4"
diagram_out = cv2.VideoWriter(diagram_output_path, fourcc, fps, (diagram.shape[1], diagram.shape[0]))

# Configuración para guardar videos a 0.5x velocidad
# Salida para videos ralentizados
slow_output_path_frame = f"./output/{video_name}_{current_time}/video_0.5x.mp4"
slow_output_path_diagram = f"./output/{video_name}_{current_time}/diagrama_0.5x.mp4"

slow_out_frame = cv2.VideoWriter(slow_output_path_frame, fourcc, fps // 2, (diagram.shape[1], diagram.shape[0]))
slow_out_diagram = cv2.VideoWriter(slow_output_path_diagram, fourcc, fps // 2, (diagram.shape[1], diagram.shape[0]))


# VARIABLES DEL PROGRAMA
# Inicializar colores de los equipos
team_colors = {'A': np.array([0,0,0]), 'B': np.array([0,0,0])}

# Inicializar posiciones de los equipos y árbitros
team_A = []
team_B = []
referees = []

previous_team_A = []
previous_team_B = []
previous_referees = []

# Inicializar puntos de la zona
zone_points = []
zone_detected = False

previous_zone_points = []

is_right_side = None


# BUCLE PRINCIPAL
# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    diagramtmp = diagram.copy()

    # HOMOGRAFÍA
    # Cálculo de los puntos candidatos a ser esquinas de la zona
    corners, xmin, ymin = getZoneCorners(frame, segment_model)
    denormalized_corners = denormalizeCorners(corners, xmin, ymin)
    corner_groups = groupCorners(denormalized_corners)

    # Si se detectan 4 esquinas, filtrarlas y comprobar si forman un cuadrilátero
    if(len(corner_groups) == 4):
        filtered_corners, is_right_side = filterCorners(corner_groups, previous_zone_points, width, height)
        if(polygonArea(filtered_corners) >= 800):
            zone_points = filtered_corners
            zone_detected = True
            previous_zone_points = zone_points

    # Si se detecta la zona, calcular la homografía
    if zone_detected:
        homography_matrix = calculateHomography(zone_points, is_right_side)
        for point in zone_points:
            cv2.circle(frame, (int(point[0]), int(point[1])), 5, (0, 255, 0), -1)
    

    # DETECCIÓN DE JUGADORES Y ÁRBITROS
    # Realizar detección en el frame
    results = detection_model(frame)

    # Análisis de las detecciones
    for result in results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if detection_model.names[cls] in ['player', 'referee'] and conf > 0.35:

            # Calcular la posición como el punto medio del borde inferior de la bbox
            position = (int((x1 + x2) / 2), y2)
            position_label = f"x:{int(position[0])} y:{int(position[1])}"

            label = f"{detection_model.names[cls]} {conf:.2f}"

            # Si la clase es 'player', clasificar el equipo y dibujar la bbox en el frame
            if detection_model.names[cls] == 'player':

                player_roi = get_central_roi(frame, int(x1), int(y1), int(x2), int(y2))

                team = classifyTeam(player_roi)

                drawBBox(frame, x1, y1, x2, y2, team, label)

                if team == 'A':
                    team_A.append(position)
                else:
                    team_B.append(position)


            # Si la clase es 'referee', dibujar la bbox en el frame
            if detection_model.names[cls] == 'referee':
                drawBBox(frame, x1, y1, x2, y2, 'referee', label)
                referees.append(position)

            # Dibujar la posición en el frame
            drawPosition(frame, position, position_label)
    
    # DIBUJAR DETECCIONES EN EL DIAGRAMA
    if zone_detected:
        # Suavizar las posiciones basadas en el frame anterior
        smoothed_team_A = smooth_positions(team_A, previous_team_A)
        smoothed_team_B = smooth_positions(team_B, previous_team_B)
        smoothed_referees = smooth_positions(referees, previous_referees)
        drawDetections(homography_matrix, diagramtmp, smoothed_team_A, smoothed_team_B, smoothed_referees)

        # Actualizar las posiciones anteriores para la próxima iteración
        previous_team_A = smoothed_team_A
        previous_team_B = smoothed_team_B
        previous_referees = smoothed_referees

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Resultados', frame)

    # Mostrar el diagrama procesado en pantalla
    cv2.imshow('Diagrama', diagramtmp)

    # Escribir el frame procesado en el video de salida
    out.write(frame)
    diagram_out.write(diagramtmp)

    # Guardar cuadros ralentizados
    slow_out_frame.write(frame)
    slow_out_diagram.write(diagramtmp)

    # Limpiar las listas de posiciones
    team_A.clear()
    team_B.clear()
    referees.clear()

    # Esperar 1ms para salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
diagram_out.release()
slow_out_frame.release()
slow_out_diagram.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")
print(f"Diagrama procesado guardado en {diagram_output_path}")
print(f"Video a 0.5x velocidad guardado en {slow_output_path_frame} y {slow_output_path_diagram}")

[NO USADO] Método filterCorners usando heurísitca del color del pixel al que corresponde cada punto candidato a ser esquina

In [5]:
def filterCornersWithColors(corner_groups, previous_zone_points, frame_width, frame_height, frame, previous_zone_colors):
    ordered_points = []
    xmin = (frame_width, 0)
    ymin = (0, frame_height)
    xmax = (0, 0)
    ymax = (0, 0)

    # Función para verificar si un color está dentro del rango permitido (en HSV)
    def is_color_in_range(color, color_range):
        lower, upper = color_range
        return all(lower[i] <= color[i] <= upper[i] for i in range(3))

    # Filtrar colores que son muy diferentes entre sí antes de calcular la media
    def filter_outliers(colors, distance_threshold=30):
        filtered_colors = []
        for i, color in enumerate(colors):
            distances = [np.linalg.norm(color - other_color) for j, other_color in enumerate(colors) if i != j]
            avg_distance = np.mean(distances)
            if avg_distance <= distance_threshold:
                filtered_colors.append(color)
        return filtered_colors

    # Calcular el rango de colores basado en colores anteriores
    def calculate_color_range(previous_colors, margin_h=15, margin_s=40, margin_v=40):
        if not previous_colors:
            # Rango por defecto amplio para el frame inicial
            return (np.array([0, 0, 0]), np.array([180, 255, 255]))

        # Filtrar colores atípicos antes de calcular el rango
        valid_colors = filter_outliers(previous_colors)

        if len(valid_colors) == 0:  # Si todos los colores son atípicos, usa todos los colores originales
            valid_colors = previous_colors

        # Convertir a HSV y calcular el promedio
        avg_color = np.mean(valid_colors, axis=0)
        lower = np.clip(avg_color - [margin_h, margin_s, margin_v], [0, 0, 0], [180, 255, 255])
        upper = np.clip(avg_color + [margin_h, margin_s, margin_v], [0, 0, 0], [180, 255, 255])
        return (lower.astype(int), upper.astype(int))

    # Convertir el frame a HSV
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Calcular el rango de colores permitido
    color_range = calculate_color_range(previous_zone_colors)

    # Filtrar y seleccionar puntos extremos con heurística basada en color
    for i in range(len(corner_groups)):
        for j in range(len(corner_groups[i])):
            point = corner_groups[i][j]
            # Obtener el color del píxel en las coordenadas del punto
            color = frame_hsv[int(point[1]), int(point[0])]

            # Verificar si el color está dentro del rango permitido
            if not is_color_in_range(color, color_range):
                continue

            # Calcular los valores de xmin, xmax, ymin, ymax
            if point[0] < xmin[0]:
                xmin = point
            if point[1] < ymin[1]:
                ymin = point
            if point[0] > xmax[0]:
                xmax = point
            if point[1] > ymax[1]:
                ymax = point

    # Detectar en qué lado de la cancha está el aro
    is_right_side = ymin[0] > ymax[0] and xmax[0] > frame_width * 0.75

    # Lista de puntos extremos detectados
    current_points = [xmin, ymax, ymin, xmax] if is_right_side else [xmax, ymax, ymin, xmin]

    if len(previous_zone_points) > 0:
        # Verificar y ajustar los puntos basándose en la distancia a previous_zone_points
        for i, current_point in enumerate(current_points):
            previous_point = previous_zone_points[i]
            distance = np.linalg.norm(np.array(current_point) - np.array(previous_point))
            print(f"Distance between current {current_point} and previous point {previous_point}: {distance:.2f}")

            if distance < 20:
                # Mantener el punto actual
                adjusted_point = previous_point
            elif 20 <= distance <= 30:
                # Suavizado exponencial
                adjusted_point = tuple(
                    0.9 * np.array(previous_point) + 0.1 * np.array(current_point)
                )
            elif 30 <= distance <= 50:
                # Suavizado exponencial
                adjusted_point = tuple(
                    0.85 * np.array(previous_point) + 0.15 * np.array(current_point)
                )
            else:
                # Mantener el punto anterior
                adjusted_point = tuple(
                    0.7 * np.array(previous_point) + 0.3 * np.array(current_point)
                )

            ordered_points.append(adjusted_point)
    else:
        ordered_points = current_points

    # Actualizar colores previos para el próximo frame
    current_zone_colors = [
        frame_hsv[int(point[1]), int(point[0])] for point in current_points
        if is_color_in_range(frame_hsv[int(point[1]), int(point[0])], color_range)
    ]
    previous_zone_colors[:] = current_zone_colors

    return ordered_points, is_right_side